In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers,Sequential
import tensorflow_hub as hub
import os
import zipfile
import shutil
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
zip_file = '/content/archive (34).zip'
extracted_file = '/content/extracted file/'


with zipfile.ZipFile(zip_file,'r') as zip_ref:
  zip_ref.extractall(extracted_file)

In [3]:
train_dir = '/content/extracted file/Human Faces Dataset/AI-Generated Images'
val_dir = '/content/extracted file/Human Faces Dataset/Real Images'

os.makedirs(os.path.join(train_dir,'class1'),exist_ok = True)
os.makedirs(os.path.join(val_dir,'class1'),exist_ok = True)

for filename in os.listdir(train_dir):
  file_path = os.path.join(train_dir,filename)
  if os.path.isfile(file_path):
    shutil.move(file_path,os.path.join(train_dir,'class1',filename))


for filename in os.listdir(val_dir):
  file_path = os.path.join(val_dir,filename)
  if os.path.isfile(file_path):
    shutil.move(file_path,os.path.join(val_dir,'class1',filename))

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    zoom_range = 0.2
)

val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (224,224),
    batch_size = 32
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (224,224),
    batch_size = 32
)

Found 4630 images belonging to 1 classes.
Found 5000 images belonging to 1 classes.


In [6]:
model = Sequential([
    layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (224,224,3)),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(512,activation = 'relu'),
    layers.Dense(2,activation = 'softmax')
])


model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.fit(train_generator,epochs = 5,validation_data = val_generator)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


145/145 ━━━━━━━━━━━━━━━━━━━━ 75s 452ms/step - accuracy: 1.0000 - loss: 0.0202 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 63s 416ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 86s 440ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 66s 441ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 82s 441ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [7]:
from tensorflow.keras.applications import InceptionResNetV2

base_model = InceptionResNetV2(include_top = False)
base_model.trainable = False

inputs = layers.Input(shape = (224,224,3))
x = base_model(inputs,training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(2,activation = 'softmax')(x)

model = tf.keras.Model(inputs = inputs,outputs = x)

model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.fit(train_generator,epochs = 5,validation_data = val_generator)

base_model.trainable = True

model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.fit(train_generator,epochs = 3,validation_data = val_generator)

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 142s 759ms/step - accuracy: 0.9988 - loss: 0.0278 - val_accuracy: 1.0000 - val_loss: 2.4516e-05
Epoch 2/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 83s 557ms/step - accuracy: 1.0000 - loss: 3.5528e-05 - val_accuracy: 1.0000 - val_loss: 2.0660e-05
Epoch 3/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 143s 566ms/step - accuracy: 1.0000 - loss: 3.0440e-05 - val_accuracy: 1.0000 - val_loss: 1.7016e-05
Epoch 4/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 83s 555ms/step - accuracy: 1.0000 - loss: 2.5175e-05 - val_accuracy: 1.0000 - val_loss: 1.4037e-05
Epoch 5/5
145/145 ━━━━━━━━━━━━━━━━━━━━ 143s 564ms/step - accuracy: 1.0000 - loss: 2.1039e-05 - val_accuracy: 1.0000 - val_loss: 1.1619e-05
Epoch 1/3
145/145 ━━━━━━━━━━━━━━━━━━━━ 370s 1s/step - accuracy: 1.0000 - loss: 5.3902e-05 - val_accuracy: 1.0000 - val_loss: 3.4938e-04
Epoch 2/3
145/145 ━━━━━━━━━━━━━━━━━━━━ 108s 731ms/step - accuracy: 1.0000 - loss: 1.9561e-09 - val_accuracy: 1.0000 - val_